<a href="https://colab.research.google.com/github/MatteoRigoni/store_python/blob/master/store_python_matteo_rigoni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Gestione di negozio per vegani
**Requisiti**
*  Registrare nuovi prodotti, con nome, quantità, prezzo di vendita e prezzo di acquisto.
*  Elencare tutti i prodotti presenti.
*  Registrare le vendite effettuate.
*  Mostrare i profitti lordi e netti.
*  Mostrare un menu di aiuto con tutti i comandi disponibili.



> Class to manage persistance on JSON files
*  *store_products*: contains all products available in the store with key on product name.
*  *selling_products*: contains all selling items as raw list.



In [117]:
import json
from os import path

class Persistance:

  filename_store = "store_products.json"
  filename_selling = "selling_products.json"

  def load_products(self):
    if path.exists(self.filename_store):
      with open(self.filename_store, 'r') as file:
        return json.load(file)
    else:
      return {}

  def save_products(self, products):
    with open(self.filename_store, 'w') as file:
      json.dump(products, file)

  def load_sellings(self):
    if path.exists(self.filename_selling):
      with open(self.filename_selling, 'r') as file:
        return json.load(file)
    else:
      return {}


  def append_sellings(self, sellings):
    if path.exists(self.filename_selling):
      with open(self.filename_selling, 'r') as file:
          existing_sellings = json.load(file)
    else:
      existing_sellings = []

    for selling in sellings:
        selling_data = {
            'name': selling,
            'quantity': sellings[selling]['quantity'],
            'sellPrice': sellings[selling]['sellPrice'],
            'buyPrice': sellings[selling]['buyPrice']}
        existing_sellings.append(selling_data)

    if len(existing_sellings) > 0:
      with open(self.filename_selling, 'w') as file:
        json.dump(existing_sellings, file)



> Show to user all possible commands



In [118]:
def command_help():
    print("""
    aggiungi: aggiungi un prodotto al magazzino
    elenca: elenca i prodotto in magazzino
    vendita: registra una vendita effettuata
    profitti: mostra i profitti totali
    aiuto: mostra i possibili comandi
    chiudi: esci dal programma
    """)



> Add a new product or update quantity of existing one



In [119]:
def add_product():
  name = input("Nome del prodotto: ").strip()
  quantity = int(input("Quantità: ").strip())

  products = Persistance().load_products()

  if name in products:
    products[name]['quantity'] += quantity
  else:
    buyPrice = float(input("Prezzo di acquisto: ").strip())
    sellPrice = float(input("Prezzo di vendita: ").strip())
    products[name] = { 'quantity': quantity, 'buyPrice': buyPrice, 'sellPrice': sellPrice }

  Persistance().save_products(products)

  print(f"{quantity} X {name}")



> Selling of one or more products, if available in the store



In [120]:
def sell_products():
  products = Persistance().load_products()
  sellings = {}

  while True:
    name = input("Nome del prodotto: ").strip()

    if name in products:
      quantity = int(input("Quantità: ").strip())
      if products[name]['quantity'] < quantity:
        print(f"Sono disponibili solo {products[name]['quantity']} pezzi!");
      else:
        products[name]['quantity'] -= quantity
        if name in sellings:
          sellings[name]['quantity'] += quantity
        else:
          sellings[name] = { 'quantity': quantity, 'sellPrice': products[name]['sellPrice'], 'buyPrice': products[name]['buyPrice']  }
    else:
      print("Prodotto non disponibile, riprova nei prossimi giorni!")

    command = input("Aggiungere un altro prodotto? (si/no): ")
    if command == "no":
      break;

  Persistance().save_products(products)
  Persistance().append_sellings(sellings)

  if len(sellings) > 0:
    print("VENDITA REGISTRATA")

    total = 0
    for name in sellings:
      print(f"- {sellings[name]['quantity']} X {name}: € {sellings[name]['sellPrice']:.2f}")
      total += sellings[name]['quantity'] * sellings[name]['sellPrice']

    print(f"Totale: € {total:.2f}")



> Fetch of all products from store and display in tabular format



In [121]:
from tabulate import tabulate

def list_products():
  products = Persistance().load_products()
  table = [[name, details['quantity'], f"€ {details['sellPrice']:.2f}"] for name, details in products.items()]
  print(tabulate(table, headers=["Nome", "Quantità", "Prezzo Unitario"], tablefmt="grid"))

> Calculate global selling income and profit

In [122]:
def calc_profit():
  sellings = Persistance().load_sellings()

  total = 0
  for selling in sellings:
    total += selling['quantity'] * selling['sellPrice']

  profit = 0
  for selling in sellings:
    profit += selling['quantity'] * (selling['sellPrice'] - selling['buyPrice'])

  print(f"Profitto: lordo: € {total:.2f}, netto: € {profit:.2f}")



> User's interaction



In [ ]:

while True:
  try:
    command = input("Inserisci un comando: ").strip()
    match command:
      case "aiuto":
        command_help()
      case "aggiungi":
        add_product()
      case "elenca":
        list_products()
      case "vendita":
        sell_products()
      case "profitti":
        calc_profit()
      case "chiudi":
        print("Alla prossima!")
        break
      case _:
        print("Comando non valido!", "I comandi dispononibili sono:")
        command_help()
  except KeyboardInterrupt:
    pass
  except Exception as error:
    print(f"Errore non gestito, contatta l'assistenza!", error)